In [1]:
import numpy as np
import pandas as pd

In [2]:
from rich.progress import track

%load_ext rich

In [3]:
monkey_strings = """
Monkey 0:
  Starting items: 57, 58
  Operation: new = old * 19
  Test: divisible by 7
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 66, 52, 59, 79, 94, 73
  Operation: new = old + 1
  Test: divisible by 19
    If true: throw to monkey 4
    If false: throw to monkey 6

Monkey 2:
  Starting items: 80
  Operation: new = old + 6
  Test: divisible by 5
    If true: throw to monkey 7
    If false: throw to monkey 5

Monkey 3:
  Starting items: 82, 81, 68, 66, 71, 83, 75, 97
  Operation: new = old + 5
  Test: divisible by 11
    If true: throw to monkey 5
    If false: throw to monkey 2

Monkey 4:
  Starting items: 55, 52, 67, 70, 69, 94, 90
  Operation: new = old * old
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 3

Monkey 5:
  Starting items: 69, 85, 89, 91
  Operation: new = old + 7
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 7

Monkey 6:
  Starting items: 75, 53, 73, 52, 75
  Operation: new = old * 7
  Test: divisible by 2
    If true: throw to monkey 0
    If false: throw to monkey 4

Monkey 7:
  Starting items: 94, 60, 79
  Operation: new = old + 2
  Test: divisible by 3
    If true: throw to monkey 1
    If false: throw to monkey 6
""".strip()

monkey_strings = monkey_strings.split("\n\n")

In [4]:
from dataclasses import dataclass
import regex as re
from typing import Optional


@dataclass(frozen=True)
class Operation:
    operator: str
    rhs: Optional[int]

    def __call__(self, lhs):
        if self.rhs is None:
            rhs = lhs
        else:
            rhs = self.rhs

        if self.operator == "+":
            return lhs + rhs
        return lhs * rhs


@dataclass
class Monkey:
    name: int
    items: list[int]
    operation: Operation
    divisor: int
    throw_to: tuple[int, int]

    def test_item(self, item):
        return item % self.divisor == 0

    @staticmethod
    def from_string(string):
        name = int(re.search(r"Monkey (\d+):", string).group(1))
        items = [
            int(match.removesuffix(", "))
            for match in re.search(r"Starting items: (\d+,? ?)+", string).captures(1)
        ]
        operation_search = re.search(r"Operation: new = old (\+|\*) (old|\d+)", string)
        operation = Operation(
            operation_search.group(1),
            None if (rhs := operation_search.group(2)) == "old" else int(rhs),
        )
        divisor = int(re.search(r"Test: divisible by (\d+)", string).group(1))
        if_true = int(re.search(r"If true: throw to monkey (\d+)", string).group(1))
        if_false = int(re.search(r"If false: throw to monkey (\d+)", string).group(1))

        return Monkey(name, items, operation, divisor, (if_true, if_false))


monkeys = [Monkey.from_string(monkey) for monkey in monkey_strings]
monkeys = {monkey.name: monkey for monkey in monkeys}
monkeys

{
    0: Monkey(
        name=0,
        items=[57, 58],
        operation=Operation(operator='*', rhs=19),
        divisor=7,
        throw_to=(2, 3)
    ),
    1: Monkey(
        name=1,
        items=[66, 52, 59, 79, 94, 73],
        operation=Operation(operator='+', rhs=1),
        divisor=19,
        throw_to=(4, 6)
    ),
    2: Monkey(
        name=2,
        items=[80],
        operation=Operation(operator='+', rhs=6),
        divisor=5,
        throw_to=(7, 5)
    ),
    3: Monkey(
        name=3,
        items=[82, 81, 68, 66, 71, 83, 75, 97],
        operation=Operation(operator='+', rhs=5),
        divisor=11,
        throw_to=(5, 2)
    ),
    4: Monkey(
        name=4,
        items=[55, 52, 67, 70, 69, 94, 90],
        operation=Operation(operator='*', rhs=None),
        divisor=17,
        throw_to=(0, 3)
    ),
    5: Monkey(
        name=5,
        items=[69, 85, 89, 91],
        operation=Operation(operator='+', rhs=7),
        divisor=13,
        throw_to=(1, 7)
    ),
    6: Monkey(
        name=6,
        items=[75, 53, 73, 52, 75],
        operation=Operation(operator='*', rhs=7),
        divisor=2,
        throw_to=(0, 4)
    ),
    7: Monkey(
        name=7,
        items=[94, 60, 79],
        operation=Operation(operator='+', rhs=2),
        divisor=3,
        throw_to=(1, 6)
    )
}

In [5]:
from copy import deepcopy
import itertools as it


def single_round(monkeys, inspections, global_modulus):
    next_monkeys = deepcopy(monkeys)
    for monkey in next_monkeys.values():
        # Increase worry
        items = [monkey.operation(item) for item in monkey.items]
        inspections[monkey.name] += len(items)
        # Decrease worry
        # items = [item // 3 for item in items]
        items = [item % global_modulus for item in items]
        # Distribute items
        true_monkey = next_monkeys[monkey.throw_to[0]]
        false_monkey = next_monkeys[monkey.throw_to[1]]
        for item in items:
            if monkey.test_item(item):
                true_monkey.items.append(item)
            else:
                false_monkey.items.append(item)
        monkey.items = []

    return next_monkeys


all_of_the_monkeys = [monkeys]
inspections = {monkey: 0 for monkey in monkeys}
global_modulus = np.prod([monkey.divisor for monkey in monkeys.values()])
for _ in track(range(10_000)):
    all_of_the_monkeys.append(
        single_round(all_of_the_monkeys[-1], inspections, global_modulus)
    )

Output()

In [6]:
inspections

{0: 60020, 1: 7592, 2: 119998, 3: 59985, 4: 62507, 5: 57608, 6: 117503, 7: 119999}

In [7]:
230 * 221

50830